In [157]:
import pandas as pd
import numpy as np

file_path = r"C:\Users\USER\Desktop\3학년 1학기\데이터 마이닝 프로젝트\남자 원본데이터.csv"
data = pd.read_csv(file_path)

In [158]:
X = data.drop(['ID','N_FIBER',
       'N_ASH', 'N_CA', 'N_PHOS', 'N_FE', 'N_NA', 'N_K', 'N_VA', 'N_CAROT',
       'N_RETIN', 'N_B1', 'N_B2', 'N_NIAC', 'N_VITC','HW_ES_DR', 'HW_MP_A', 'HW_MS_A', 'HW_OC_DR','N_WAT_C'], axis=1)  
# 타겟 변수, string타입의 id삭제, 필요없다고 생각되는 비타민..., 여자 columnn삭제

In [159]:
# 데이터프레임에서 결측치 있는 특성 확인
missing_columns= X.columns[X.isnull().any()]

list(missing_columns)

['N_DIET',
 'N_INTK',
 'N_EN',
 'N_WATER',
 'N_PROT',
 'N_FAT',
 'N_CHO',
 'L_LN_FAM',
 'DW_WBT_BMD']

1. 'L_LN_FAM' (가족과 저녁식사 횟수) 결측치 처리

-> 비율에 맞춰서 결측치 처리

In [160]:
col = 'L_LN_FAM'
#결측치 개수 출력 (isnull().sum())
total_miss = X[col].isnull().sum()
print("결측치 개수: ", total_miss)

# 각 값의 개수 출력
print(X[col].value_counts())

결측치 개수:  277
2.0    748
1.0    408
Name: L_LN_FAM, dtype: int64


In [161]:
# 값이 1, 2인 개수 (loc은 이름 or 조건, iloc은 인덱스)
x1=len(X.loc[X['L_LN_FAM']==1])
x2=len(X.loc[X['L_LN_FAM']==2])
total =x1+x2

r1=x1/total
r2=x2/total
r_list=[]

#r_list에 결측치 값을 비율에 맞춰서 추가
for i in range(round(r1*total_miss)):   #정수값을 넣기위해 round사용
    r_list.append(1)

for i in range(round(r2*total_miss)):   #정수값을 넣기위해 round사용
    r_list.append(2)

In [162]:
import random
random.shuffle(r_list)

#x데이터프레임의 'L_LN_FAM'의 결측치있는 행번호로 리스트 생성
na_index= X[X['L_LN_FAM'].isnull()].index.to_list()

#list에 무작위로 저장되어있는 값을 결측치가 있는 행에 하나씩 채워넣음
for i in range(total_miss):
    X.loc[na_index[i], 'L_LN_FAM'] = r_list[i]  #loc에서 앞은 행, 뒤는 열


In [163]:
col = 'L_LN_FAM'
total_miss = X[col].isnull().sum()
print("결측치 개수: ", total_miss)

결측치 개수:  0


'L_LN_FAM' 결측치 처리 완료

2. 'N_DIET' (식이요법 여부)

In [164]:
col = 'N_DIET'
#결측치 개수 출력 (isnull().sum())
total_miss = X[col].isnull().sum()
print("결측치 개수: ", total_miss)

# 각 값의 개수 출력
print(X[col].value_counts())

결측치 개수:  279
2.0    949
1.0    205
Name: N_DIET, dtype: int64


In [165]:
# 값이 1, 2인 개수 (loc은 이름 or 조건, iloc은 인덱스)
x1=len(X.loc[X['N_DIET']==1])
x2=len(X.loc[X['N_DIET']==2])
total =x1+x2

r1=x1/total
r2=x2/total
r_list=[]

#r_list에 결측치 값을 비율에 맞춰서 추가
for i in range(round(r1*total_miss)):   #정수값을 넣기위해 round사용
    r_list.append(1)

for i in range(round(r2*total_miss)):   #정수값을 넣기위해 round사용
    r_list.append(2)

In [166]:
import random
random.shuffle(r_list)

#x데이터프레임의 'L_LN_FAM'의 결측치있는 행번호로 리스트 생성
na_index= X[X['N_DIET'].isnull()].index.to_list()

#list에 무작위로 저장되어있는 값을 결측치가 있는 행에 하나씩 채워넣음
for i in range(total_miss):
    X.loc[na_index[i], 'N_DIET'] = r_list[i]  #loc에서 앞은 행, 뒤는 열


In [167]:
col = 'N_DIET'
total_miss = X[col].isnull().sum()
print("결측치 개수: ", total_miss)

결측치 개수:  0


N_DIET 결측치 처리 완료

카테고리형 결측치 처리 완료

---------------------------------

연속형 결측치 처리

In [168]:
# X 데이터프레임에서 결측치가 있는 특성 확인
missing_columns = X.columns[X.isnull().any()]

missing_columns

Index(['N_INTK', 'N_EN', 'N_WATER', 'N_PROT', 'N_FAT', 'N_CHO', 'DW_WBT_BMD'], dtype='object')

In [169]:
cor_list=[]
for i in list(missing_columns):
    impute_feature = i # 선형 대치할 대상 특성

    # 대상 특성과 각 특성들과의 상관계수(절대값) 계산 -> series로 저장
    correlations = X.corr()[impute_feature].abs()
    #대상 특성은 제외
    correlations.drop(impute_feature, inplace=True)
    #상관계수가 0.5이상인 인덱스만 list로 저장
    selected_features = correlations[correlations >= 0.5].index.tolist()
    print(f"상관계수 0.5이상: {impute_feature}: {selected_features}")
    cor_list.append(selected_features)


상관계수 0.5이상: N_INTK: ['N_EN', 'N_WATER', 'N_PROT', 'N_FAT', 'N_CHO']
상관계수 0.5이상: N_EN: ['N_INTK', 'N_WATER', 'N_PROT', 'N_FAT', 'N_CHO']
상관계수 0.5이상: N_WATER: ['N_INTK', 'N_EN', 'N_PROT']


상관계수 0.5이상: N_PROT: ['N_INTK', 'N_EN', 'N_WATER', 'N_FAT']
상관계수 0.5이상: N_FAT: ['N_INTK', 'N_EN', 'N_PROT']
상관계수 0.5이상: N_CHO: ['N_INTK', 'N_EN']
상관계수 0.5이상: DW_WBT_BMD: []


In [170]:
# from sklearn.linear_model import LinearRegression

# for i, selected_features in zip(missing_columns, cor_list):
#     # 대치할 행 선택
#     missing_rows = X[i].isnull()

#     # 대치할 목표 특성과 상관계수가 높게 측정된 특성 선택
#     impute_target = X[i]
#     impute_features = X[selected_features]

#     # 결측치 대치를 위한 선형 회귀 모델 훈련
#     model = LinearRegression()
#     model.fit(impute_features[~missing_rows], impute_target[~missing_rows])

#     # 결측치 대치
#     imputed_values = model.predict(impute_features[missing_rows])
#     X.loc[missing_rows, i] = imputed_values


선형관계가 유의미한 컬럼은 모두 연속형이므로 선형대치 불가

대안으로 knn대치 선택

In [171]:
from sklearn.impute import KNNImputer

# 결측치가 있는 특성 선택
impute_features = ['N_INTK', 'N_EN', 'N_WATER', 'N_PROT', 'N_FAT', 'N_CHO',
       'DW_WBT_BMD']
# KNNImputer 객체 생성
imputer = KNNImputer(n_neighbors=5)  # 임의로 이웃은 5명으로 설정

# KNN 대치 수행
X_imputed = imputer.fit_transform(X[impute_features])

# 대치된 값으로 열을 갱신
X[impute_features] = X_imputed

In [172]:
X.columns[X.isnull().any()]

Index([], dtype='object')

결측치 처리 완료